### Data Cleaning 

In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
df=pd.read_csv("deptos_mercado_libre_full.csv", index_col=0)

In [3]:
df.head(3)

,titulos,superficie,recamaras,precio,ubicacion,imagen
0,"Súper Ubicado, Moderno, Metrobús, Escuelas, Ho...",119 m² construidos,3 recs.,20000.0,"['Gabriel Mancera', 'Del Valle Norte', 'Benito...",https://http2.mlstatic.com/D_NQ_NP_866287-MLM4...
1,"Departamento Amueblado En Zakia Residencial, Q...",120 m² construidos,3 recs.,14800.0,"['Paseo De Zinnia 201', 'Zákia', 'El Marqués',...",https://http2.mlstatic.com/D_NQ_NP_693487-MLM4...
2,Hermoso Departamento En Renta Amueblado En Ag...,73 m² construidos,2 recs.,27000.0,"['Av. Insurgentes Sur', 'Loreto', 'Alvaro Obre...",https://http2.mlstatic.com/D_NQ_NP_891309-MLM4...


In [4]:
df.shape

(1104, 6)

In [5]:
df["ubicacion"][6]

"['Departamento En Renta Sobre Boulevard 5 De Mayo! Increible Vista Panoramica!', 'Huexotitla', 'Puebla']"

In [6]:
#df["ubicacion"][6]=ast.literal_eval(df["ubicacion"][6])
#df["ubicacion"][6]

In [7]:
for i in range(len(df["ubicacion"])):
    df["ubicacion"][i]=ast.literal_eval(df["ubicacion"][i])   

/tmp/ipykernel_10172/1645389848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ubicacion"][i]=ast.literal_eval(df["ubicacion"][i])


In [8]:
df["ubicacion"][6]

['Departamento En Renta Sobre Boulevard 5 De Mayo! Increible Vista Panoramica!',
 'Huexotitla',
 'Puebla']

No todas las direcciones tienen 4 elementos ( calle, colonia, delegación o municipio y estado). Por lo que tenemos dos problemas a resolver; casos con más información y con menos. 

In [9]:
direccion=df["ubicacion"]

In [10]:
j = 0
mas_info_indices=[]
for i in range(len(direccion)):
    if (len(direccion[i]) > 4):
        j+=1
        
        mas_info_indices.append(i)
print("Hay",j, "direcciones con más información")
mas_info_indices;

Hay 92 direcciones con más información


In [11]:
direccion[12]

['Magnolia 56', 'Depto. 3', 'Los Morales', 'Cuautitlán', 'Estado De México']

In [12]:
direccion[25]

['Av. Emiliano Zapata (eje 7a Sur) 360',
 'Departamento 301',
 'Torre B',
 'Edif. 1',
 'Santa Cruz Atoyac',
 'Benito Juárez',
 'Distrito Federal']

In [13]:
k = 0
menos_info_indices=[]
for i in range(len(direccion)):
    if (len(direccion[i]) < 4):
        j+=1
        
        menos_info_indices.append(i)
print("Hay",k, "direcciones con menos información")
menos_info_indices;

Hay 0 direcciones con menos información


In [14]:
(161+92)/len(direccion)*100

22.916666666666664

In [15]:
direccion[6]

['Departamento En Renta Sobre Boulevard 5 De Mayo! Increible Vista Panoramica!',
 'Huexotitla',
 'Puebla']

In [16]:
direccion[27]

['Calzada Manuel Villalongin 213 Int. 205', 'Cuauhtémoc', 'Distrito Federal']

Podemos seleccionar una calle o una colonia de la siguiente forma:

In [17]:
# Calle
direccion[0][0]

'Gabriel Mancera'

In [18]:
# Colonia
direccion[0][1]

'Del Valle Norte'

A continuación separaresemos la información contenida en cada dirección en 4 listas: calle, colonia, municipio y estado. 

Para resolver el caso en el que tenemos menos información,  dependiendo del tamaño de la lista de dirección, vamos a llenar la información faltante con NaN's. Notemos que en estos casos no se suele omitir el nombre de la calle, por lo tanto no reemplazaremos el primer elemento de las listas por un NaN.   

En el caso en le que tenemos más información, notamos que se debe a que se añaden datos que mejoran la precisión de la ubicación, por lo tanto podemos incluir dichos datos dentro de la información de las calles.  Para ello uniremos los primeros elementos de las listas, tomando en cuenta que los últimos 3 de cada lista son colonia, municipio y estado.

In [19]:
calle = []
colonia = []
municipio = []
estado = []

j=0
for i in range(len(direccion)):
    if (len(direccion[i]) == 4):
        
        calle.append(direccion[i][0])
        colonia.append(direccion[i][1])
        municipio.append(direccion[i][2])
        estado.append(direccion[i][3])
        
    elif(len(direccion[i])==3):

        direccion[i].insert(1,np.nan) #suponiendo que no se suele omitir la calle
        
        calle.append(direccion[i][0])
        colonia.append(direccion[i][1])
        municipio.append(direccion[i][2])
        estado.append(direccion[i][3])
        
        
    elif(len(direccion[i])==2):
        
        direccion[i].insert(1,np.nan)
        direccion[i].insert(2,np.nan) 

        calle.append(direccion[i][0])
        colonia.append(direccion[i][1])
        municipio.append(direccion[i][2])
        estado.append(direccion[i][3])


    elif(len(direccion[i])==1):
        direccion[i].insert(1,np.nan)
        direccion[i].insert(2,np.nan)
        direccion[i].insert(3,np.nan) 


        calle.append(direccion[i][0])
        colonia.append(direccion[i][1])
        municipio.append(direccion[i][2])
        estado.append(direccion[i][3])
        
    elif (len(direccion[i]) > 4):

        #uniremos los primeros elementos de las listas.
        #tomando en cuenta que los últimos 3 de cada lista son colonia, municipio y estado.
        direccion[i][0:len(direccion[i])-3] = [' '.join(direccion[i][0:len(direccion[i])-3])]
        
        calle.append(direccion[i][0])
        colonia.append(direccion[i][1])
        municipio.append(direccion[i][2])
        estado.append(direccion[i][3])
    

In [20]:
a=27

print("Calle:",calle[a],"\n","Colonia:",colonia[a],"\n","Municipio:",municipio[a],"\n","Estado:",estado[a])

Calle: Calzada Manuel Villalongin 213 Int. 205 
 Colonia: nan 
 Municipio: Cuauhtémoc 
 Estado: Distrito Federal


In [21]:
a=6

print("Calle:",calle[a],"\n","Colonia:",colonia[a],"\n","Municipio:",municipio[a],"\n","Estado:",estado[a])

Calle: Departamento En Renta Sobre Boulevard 5 De Mayo! Increible Vista Panoramica! 
 Colonia: nan 
 Municipio: Huexotitla 
 Estado: Puebla


In [22]:
a=12

print("Calle:",calle[a],"\n","Colonia:",colonia[a],"\n","Municipio:",municipio[a],"\n","Estado:",estado[a])

Calle: Magnolia 56 Depto. 3 
 Colonia: Los Morales 
 Municipio: Cuautitlán 
 Estado: Estado De México


In [23]:
a=25

print("Calle:",calle[a],"\n","Colonia:",colonia[a],"\n","Municipio:",municipio[a],"\n","Estado:",estado[a])

Calle: Av. Emiliano Zapata (eje 7a Sur) 360 Departamento 301 Torre B Edif. 1 
 Colonia: Santa Cruz Atoyac 
 Municipio: Benito Juárez 
 Estado: Distrito Federal


Añadimos las nuevas columnas al DataFrame

In [24]:
df['calle'] = calle
df['colonia'] = colonia
df['municipio'] = municipio
df['estado'] = estado

In [25]:
df.head()

,titulos,superficie,recamaras,precio,ubicacion,imagen,calle,colonia,municipio,estado
0,"Súper Ubicado, Moderno, Metrobús, Escuelas, Ho...",119 m² construidos,3 recs.,20000.0,"[Gabriel Mancera, Del Valle Norte, Benito Juár...",https://http2.mlstatic.com/D_NQ_NP_866287-MLM4...,Gabriel Mancera,Del Valle Norte,Benito Juárez,Distrito Federal
1,"Departamento Amueblado En Zakia Residencial, Q...",120 m² construidos,3 recs.,14800.0,"[Paseo De Zinnia 201, Zákia, El Marqués, Queré...",https://http2.mlstatic.com/D_NQ_NP_693487-MLM4...,Paseo De Zinnia 201,Zákia,El Marqués,Querétaro
2,Hermoso Departamento En Renta Amueblado En Ag...,73 m² construidos,2 recs.,27000.0,"[Av. Insurgentes Sur, Loreto, Alvaro Obregón, ...",https://http2.mlstatic.com/D_NQ_NP_891309-MLM4...,Av. Insurgentes Sur,Loreto,Alvaro Obregón,Distrito Federal
3,Confortable Departamento Con Privacidad Y Exce...,61 m² construidos,2 recs.,11400.0,"[Rafael Martínez Rip Rip, Independencia, Benit...",https://http2.mlstatic.com/D_NQ_NP_799707-MLM4...,Rafael Martínez Rip Rip,Independencia,Benito Juárez,Distrito Federal
4,Departamento En Renta Coyoacán Col. Parque San...,52 m² construidos,2 recs.,13000.0,"[Carreteraco, Parque San Andrés, Coyoacán, Dis...",https://http2.mlstatic.com/D_NQ_NP_649773-MLM4...,Carreteraco,Parque San Andrés,Coyoacán,Distrito Federal


De los datos que añadimos, los más importantes son estado, municipio/alcaldía y colonia. Para estas columnas tenemos que asegurarnos que su redacción sea correcta, para evitar duplicados debidos a diferencias en la escritura (uso de mayúsculas, acentos, errores ortográficos, etc). Notemos que con estos datos nos será posible obtener los códigos postales con los cuales podremos ubicar los departamentos en un mapa. 

Por otro lado, nos será conveniente para cálculos tener solo valores numéricos en las columnas de superficie y recámaras. 

In [26]:
df["estado"]

0       Distrito Federal
1              Querétaro
2       Distrito Federal
3       Distrito Federal
4       Distrito Federal
              ...       
1099    Estado De México
1100    Distrito Federal
1101        Quintana Roo
1102    Distrito Federal
1103    Distrito Federal
Name: estado, Length: 1104, dtype: object

In [27]:
estados=df["estado"].dropna().unique()
estados

array(['Distrito Federal', 'Querétaro', 'Quintana Roo', 'Puebla',
       'Estado De México', 'Nuevo León', 'Baja California', 'Jalisco'],
      dtype=object)

In [28]:
municipios=df["municipio"].dropna().unique()
municipios

array(['Benito Juárez', 'El Marqués', 'Alvaro Obregón', 'Coyoacán',
       'Cancún/benito Juárez', 'Huexotitla', 'Gustavo A. Madero',
       'Cuautitlán', 'Azcapotzalco', 'Solidaridad/riviera Maya',
       'Santa Catarina', 'Atizapán De Zaragoza', 'San Juan Del Río',
       'Coacalco De Berriozabal', 'Tijuana', 'Miguel Hidalgo',
       'Cuauhtémoc', 'Toluca', 'Huixquilucan', 'Guadalajara', 'Naucalpan'],
      dtype=object)

In [29]:
colonias=df["colonia"].dropna().unique()
colonias

array(['Del Valle Norte', 'Zákia', 'Loreto', 'Independencia',
       'Parque San Andrés', 'Cancún Centro', 'Nápoles',
       'Residencial Zacatenco', 'Granjas Modernas', 'Albert',
       'Los Morales', 'Clavería', 'Centro',
       'Valle Poniente Sector Olinca', 'México Nuevo', 'Ejidal',
       'Del Gas', 'Valle De Oro', 'Lomas De Coacalco',
       'Lomas De San Lorenzo', 'Narvarte Poniente',
       'Zona Urbana Río Tijuana', 'Santa Cruz Atoyac', 'Irrigación',
       'Guerrero', 'Industrial San Antonio', 'Supermanzana 523',
       'Granada', 'Industrial', 'Jesús Del Monte', 'Insurgentes Mixcoac',
       'Espartaco', 'Ladrón De Guevara', 'Supermanzana 97',
       'Santa María Magdalena', 'Bosque Alto', 'Playa Del Carmen Centro',
       'San Pedro Xalpa'], dtype=object)

In [30]:
from similares import replace_matches_in_column
from similares import busca_similares
#replace_matches_in_column?
busca_similares?

/home/solaris/anaconda3/envs/cursos/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Signature: busca_similares(columna_unique, cantidad)
Docstring:
Example: 
Input: Columna de un dataframe al que se le ha aplicado el método .unique()
       y una "cantidad" de coincidencias a mostrar por palabra.  

Output: Las strings más cercanas entre sí de toda la columna. 
File:      ~/Documentos/cursos_datos/proyectos/Mercado_Libre/Data_Cleaning/similares.py
Type:      function


In [31]:
busca_similares(municipios,3)

0 [('Benito Juárez', 100), ('Cancún/benito Juárez', 80), ('Miguel Hidalgo', 38)]
1 [('El Marqués', 100), ('San Juan Del Río', 50), ('Huexotitla', 32)]
2 [('Alvaro Obregón', 100), ('Santa Catarina', 37), ('Miguel Hidalgo', 37)]
3 [('Coyoacán', 100), ('Cuautitlán', 38), ('Cuauhtémoc', 38)]
4 [('Cancún/benito Juárez', 100), ('Benito Juárez', 80), ('Toluca', 33)]
5 [('Huexotitla', 100), ('Huixquilucan', 55), ('Cuautitlán', 53)]
6 [('Gustavo A. Madero', 100), ('Miguel Hidalgo', 40), ('Guadalajara', 37)]
7 [('Cuautitlán', 100), ('Cuauhtémoc', 56), ('Huexotitla', 53)]
8 [('Azcapotzalco', 100), ('Naucalpan', 48), ('Atizapán De Zaragoza', 45)]
9 [('Solidaridad/riviera Maya', 100), ('Santa Catarina', 47), ('Atizapán De Zaragoza', 37)]
10 [('Santa Catarina', 100), ('Cuautitlán', 43), ('San Juan Del Río', 41)]
11 [('Atizapán De Zaragoza', 100), ('Gustavo A. Madero', 46), ('Tijuana', 38)]
12 [('San Juan Del Río', 100), ('Tijuana', 45), ('Alvaro Obregón', 43)]
13 [('Coacalco De Berriozabal', 100), (

In [32]:
busca_similares(municipios,3)

0 [('Benito Juárez', 100), ('Cancún/benito Juárez', 80), ('Miguel Hidalgo', 38)]
1 [('El Marqués', 100), ('San Juan Del Río', 50), ('Huexotitla', 32)]
2 [('Alvaro Obregón', 100), ('Santa Catarina', 37), ('Miguel Hidalgo', 37)]
3 [('Coyoacán', 100), ('Cuautitlán', 38), ('Cuauhtémoc', 38)]
4 [('Cancún/benito Juárez', 100), ('Benito Juárez', 80), ('Toluca', 33)]
5 [('Huexotitla', 100), ('Huixquilucan', 55), ('Cuautitlán', 53)]
6 [('Gustavo A. Madero', 100), ('Miguel Hidalgo', 40), ('Guadalajara', 37)]
7 [('Cuautitlán', 100), ('Cuauhtémoc', 56), ('Huexotitla', 53)]
8 [('Azcapotzalco', 100), ('Naucalpan', 48), ('Atizapán De Zaragoza', 45)]
9 [('Solidaridad/riviera Maya', 100), ('Santa Catarina', 47), ('Atizapán De Zaragoza', 37)]
10 [('Santa Catarina', 100), ('Cuautitlán', 43), ('San Juan Del Río', 41)]
11 [('Atizapán De Zaragoza', 100), ('Gustavo A. Madero', 46), ('Tijuana', 38)]
12 [('San Juan Del Río', 100), ('Tijuana', 45), ('Alvaro Obregón', 43)]
13 [('Coacalco De Berriozabal', 100), (

In [33]:
busca_similares(colonias,3)

0 [('Del Valle Norte', 100), ('Valle De Oro', 81), ('Los Morales', 54)]
1 [('Zákia', 100), ('Ejidal', 40), ('Irrigación', 31)]
2 [('Loreto', 100), ('Albert', 50), ('Los Morales', 47)]
3 [('Independencia', 100), ('Insurgentes Mixcoac', 44), ('Industrial', 43)]
4 [('Parque San Andrés', 100), ('Lomas De San Lorenzo', 50), ('Santa María Magdalena', 50)]
5 [('Cancún Centro', 100), ('Centro', 67), ('Playa Del Carmen Centro', 57)]
6 [('Nápoles', 100), ('Los Morales', 47), ('Del Valle Norte', 38)]
7 [('Residencial Zacatenco', 100), ('Independencia', 41), ('Espartaco', 40)]
8 [('Granjas Modernas', 100), ('Granada', 61), ('Los Morales', 44)]
9 [('Albert', 100), ('Loreto', 50), ('Bosque Alto', 47)]
10 [('Los Morales', 100), ('Del Valle Norte', 54), ('Granjas Modernas', 52)]
11 [('Clavería', 100), ('Albert', 46), ('Centro', 46)]
12 [('Centro', 100), ('Cancún Centro', 67), ('Loreto', 50)]
13 [('Valle Poniente Sector Olinca', 100), ('Valle De Oro', 50), ('Narvarte Poniente', 49)]
14 [('México Nuevo'

In [34]:
busca_similares(estados,3)

0 [('Distrito Federal', 100), ('Estado De México', 32), ('Quintana Roo', 29)]
1 [('Querétaro', 100), ('Quintana Roo', 60), ('Puebla', 43)]
2 [('Quintana Roo', 100), ('Querétaro', 60), ('Estado De México', 30)]
3 [('Puebla', 100), ('Querétaro', 43), ('Baja California', 29)]
4 [('Estado De México', 100), ('Querétaro', 35), ('Nuevo León', 33)]
5 [('Nuevo León', 100), ('Querétaro', 35), ('Estado De México', 33)]
6 [('Baja California', 100), ('Jalisco', 45), ('Quintana Roo', 30)]
7 [('Jalisco', 100), ('Baja California', 45), ('Estado De México', 36)]


In [35]:
from similares import replace_matches_in_column
replace_matches_in_column?

Signature: replace_matches_in_column(df, column, string_to_match, min_ratio=47)
Docstring:
Example: Replace close matches to "southkorea" with "south korea":

replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")
File:      ~/Documentos/cursos_datos/proyectos/Mercado_Libre/Data_Cleaning/similares.py
Type:      function


In [36]:
#Cancún/benito Juárez->Benito Juárez

replace_matches_in_column(df=df, column='municipio', string_to_match="Benito Juárez")




All done!


In [37]:
df["municipio"][15]=="Solidaridad/riviera Maya"

True

In [38]:
df["municipio"]=df["municipio"].replace(["Solidaridad/riviera Maya"], "Solidaridad")

In [39]:
df["municipio"][15]

'Solidaridad'

In [40]:
df["superficie"][0]

'119 m² construidos'

In [41]:
a=df["superficie"][0]

In [42]:
b=a.replace(" m² construidos","")
float(b)

119.0

In [43]:
#probar un , inplace=True para quitar la advertencia

for i  in range(len(df["superficie"])):
    df["superficie"][i]=df["superficie"][i].replace(" m² construidos","")

/tmp/ipykernel_10172/1926137496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["superficie"][i]=df["superficie"][i].replace(" m² construidos","")


In [44]:
for i  in range(len(df["superficie"])):
    df["superficie"][i]=float(df["superficie"][i])

/tmp/ipykernel_10172/3769706909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["superficie"][i]=float(df["superficie"][i])


In [45]:
for i  in range(len(df["recamaras"])):
    df["recamaras"].loc[i]=df["recamaras"][i].replace(" recs.","")

/tmp/ipykernel_10172/2904285959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["recamaras"].loc[i]=df["recamaras"][i].replace(" recs.","")


In [46]:
for i  in range(len(df["recamaras"])):
    df["recamaras"].loc[i]=df["recamaras"].loc[i].replace(" rec.","")

/tmp/ipykernel_10172/1948828721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["recamaras"].loc[i]=df["recamaras"].loc[i].replace(" rec.","")


In [47]:
for i  in range(len(df["recamaras"])):
    df["recamaras"].loc[i]=int(df["recamaras"].loc[i])

/tmp/ipykernel_10172/465487925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["recamaras"].loc[i]=int(df["recamaras"].loc[i])


In [48]:
df.head()

,titulos,superficie,recamaras,precio,ubicacion,imagen,calle,colonia,municipio,estado
0,"Súper Ubicado, Moderno, Metrobús, Escuelas, Ho...",119.0,3,20000.0,"[Gabriel Mancera, Del Valle Norte, Benito Juár...",https://http2.mlstatic.com/D_NQ_NP_866287-MLM4...,Gabriel Mancera,Del Valle Norte,Benito Juárez,Distrito Federal
1,"Departamento Amueblado En Zakia Residencial, Q...",120.0,3,14800.0,"[Paseo De Zinnia 201, Zákia, El Marqués, Queré...",https://http2.mlstatic.com/D_NQ_NP_693487-MLM4...,Paseo De Zinnia 201,Zákia,El Marqués,Querétaro
2,Hermoso Departamento En Renta Amueblado En Ag...,73.0,2,27000.0,"[Av. Insurgentes Sur, Loreto, Alvaro Obregón, ...",https://http2.mlstatic.com/D_NQ_NP_891309-MLM4...,Av. Insurgentes Sur,Loreto,Alvaro Obregón,Distrito Federal
3,Confortable Departamento Con Privacidad Y Exce...,61.0,2,11400.0,"[Rafael Martínez Rip Rip, Independencia, Benit...",https://http2.mlstatic.com/D_NQ_NP_799707-MLM4...,Rafael Martínez Rip Rip,Independencia,Benito Juárez,Distrito Federal
4,Departamento En Renta Coyoacán Col. Parque San...,52.0,2,13000.0,"[Carreteraco, Parque San Andrés, Coyoacán, Dis...",https://http2.mlstatic.com/D_NQ_NP_649773-MLM4...,Carreteraco,Parque San Andrés,Coyoacán,Distrito Federal


In [49]:
#df.drop("Unnamed: 0", inplace=True, axis=1)


In [50]:
df.to_csv("deptos_mercado_libre_full_clean.csv")

---

# Carga de datos con API de SQL

In [ ]:
import postgres
import psycopg2

Información de la base en ElephantSQL: https://api.elephantsql.com/console/cc7a0605-0a78-4657-8047-3f9d479faba7/details

### Conector

In [ ]:
config = {'host':'castor.db.elephantsql.com',
                             'user':'xidnzmgq',
                             'password':'x2vRkaLOyY6Fxzid-W0mzsE001tMmeRE',
                             'database':'xidnzmgq'}

def get_connection(config):

    return psycopg2.connect(**config)

### Creamos la tabla

In [ ]:
conn= get_connection(config)


# Creamos una consulta para generar la tabla 

create_table_deptos = "CREATE TABLE DataDeptos (titulos varchar(50),superficie int(50),recamaras int(50),precio int(50),ubicacion varchar(50),calle varchar(50), colonia varchar(50),municipio varchar(50),estado varchar(50))"

# Necesitamos enviar la orden/consulta anterior por medio de un cursor. 

db_cursor = conn.cursor()

# Ejecutamos la consulta

db_cursor.execute(create_table_deptos)


# Guardamos los cambios

conn.commit()
# Cerramos
db_cursor.close()
conn.close()

## Introducción de los datos

In [ ]:
def insert_depto(df):
    
    #Abrimos las conecciones
    
    conn= get_connection(config)

    db_cursor = conn.cursor()

    # Consulta para insertar los datos

    insert_query = "INSERT INTO UserData (titulos, superficie, recamaras, precio, ubicacion, calle, colonia, municipio, estado) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    
    for i in range(len(df.index)):
        
        parametros = (df.iloc[i][0],df.iloc[i][1],df.iloc[i][2])
        db_cursor.execute(insert_query,parametros)

    # Guardamos los cambios

    conn.commit()

    # Cerramos

    db_cursor.close()
    conn.close()